# Предсказание стоимости жилья

В проекте нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Подготовка данных

Сначала произведем все необходимые импорты:

In [1]:
import pandas as pd 
import numpy as np
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import seaborn as sns
from pyspark.ml.feature import OneHotEncoder 
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
RANDOM_SEED = 2022

Инициализируем локальную Spark-сессию:

In [2]:
spark = SparkSession.builder \
                    .appName("EDA California Housing") \
                    .getOrCreate()

Прочитаем содержимое файла `'\/datasets/housing.csv'`:

In [3]:
df_housing = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 
df_housing.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



В колонках датасета содержатся следующие данные:

* `longitude` — широта;
* `latitude` — долгота;
* `housing_median_age` — медианный возраст жителей жилого массива;
* `total_rooms` — общее количество комнат в домах жилого массива;
* `total_bedrooms` — общее количество спален в домах жилого массива;
* `population` — количество человек, которые проживают в жилом массиве;
* `households` — количество домовладений в жилом массиве;
* `median_income` — медианный доход жителей жилого массива;
* `median_house_value` — медианная стоимость дома в жилом массиве;
* `ocean_proximity` — близость к океану.

В большинстве колонок хранятся количественные данные, кроме одной — ocean_proximity. Она хранит категориальные значения.

Выведем типы данных колонок датасета. Используем для этого метод pySpark `dtypes`:

In [4]:
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


Более детально изучим данные. Для этого вызовем метод `describe`:

In [5]:
df_housing.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Выполним предобработку данных.

Исследуем данные на наличие пропусков:

In [6]:
columns = df_housing.columns


for column in columns:
    print(column, df_housing.where(F.isnan(column) | F.col(column).isNull()).count())
    

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Колонка `total_bedrooms` содержит 207 пропусков - это 1% от общего количества записей в таблице. Поэтому удалим строки, содержащие пропуски:

In [7]:
df_housing = df_housing.na.drop(how='any')

Проверим, как сработало удаление:

In [8]:
columns = df_housing.columns


for column in columns:
    print(column, df_housing.where(F.isnan(column) | F.col(column).isNull()).count())
    

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


Разделим колонки на два типа: числовые и текстовые, которые представляют категориальные данные:

In [9]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
target = "median_house_value" 


Далее приступим к первому этапу — трансформации признаков.

Преобразуем колонку с категориальными значениями с помощью базовых трансформеров признаков — StringIndexer и OneHotEncoder:

Сначала трансформируем категориальные признаки в числовое представление с помощью трансформера StringIndexer:

In [10]:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
df_housing = indexer.fit(df_housing).transform(df_housing)

Далее выполним OHE-кодирование для категорий:

In [11]:
ohe = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])

df_housing = ohe.fit(df_housing).transform(df_housing)
df_housing.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,ocean_proximity_idx,ocean_proximity_ohe
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)"
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)"
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)"
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)"
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,3.0,"(0.0, 0.0, 0.0, 1.0)"


Объединим  признаки в один вектор с помощью VectorAssembler:

In [12]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
df_housing = categorical_assembler.transform(df_housing)

## Обучение моделей

Прежде, чем перейти к обучению моделей мы трансформируем и числовые признаки. Это нужно, чтобы сильные выбросы не влияли на результат работы модели.

In [13]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
df_housing = numerical_assembler.transform(df_housing)
standardScaler = StandardScaler(inputCol='numerical_features',outputCol="numerical_features_scaled")
df_housing = standardScaler.fit(df_housing).transform(df_housing) 

Далее соберем полученные категорийные и числовые признаки с помощью VectorAssembler:

In [14]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, outputCol="features") 
df_housing = final_assembler.transform(df_housing)

In [15]:
df_housing.select(all_features).show(3) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[3],[1.0])|     [-61.005863841998...|
|       (4,[3],[1.0])|     [-61.000872770752...|
|       (4,[3],[1.0])|     [-61.010854913244...|
+--------------------+-------------------------+
only showing top 3 rows



Нам нужно построить две модели линейной регрессии на разных наборах данных. 

Для этого разделим наш датасет на две части  - выборку для обучения и выборку для тестирования качества модели.
Для каждого разбиения будем использовать  метод `randomSplit()`.Выведем количество записей в выборках на экран при помощи метода `count()`:

In [16]:
train_data, test_data = df_housing.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16247 4186


Первая модель линейной регрессии будет использовать все данные:

In [17]:
lr_1 = LinearRegression(labelCol=target, featuresCol='features',regParam=0.0)

In [18]:
model = lr_1.fit(train_data) 

22/03/31 07:45:13 WARN Instrumentation: [1b1fb85d] regParam is zero, which might cause numerical instability and overfitting.
22/03/31 07:45:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/03/31 07:45:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/03/31 07:45:14 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/03/31 07:45:14 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


Сохраним трансформированную таблицу с колонкой предсказания первой модели в переменной `predictions`:

In [19]:
predictions = model.transform(test_data)

In [20]:
predictions_col = 'prediction'

Теперь аналогично построим вторую модель, используя только числовые переменные, исключив категориальные:

In [21]:
lr_2 = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled', regParam=0.0)

In [22]:
model2 = lr_2.fit(train_data) 

22/03/31 07:45:17 WARN Instrumentation: [af513d5e] regParam is zero, which might cause numerical instability and overfitting.


Сохраним трансформированную таблицу с колонкой предсказания первой модели в переменной `predictions2`:

In [23]:
predictions2 = model2.transform(test_data)

## Анализ результатов

Для анализа результатов работы модели создадим объект класса RegressionEvaluator:

In [24]:
evaluator = RegressionEvaluator(predictionCol=predictions_col, labelCol = target)

Сравним результаты работы линейной регрессии на двух наборах данных по следующим метрикам:

### Корень от средней квадратической ошибки - RMSE: 

Первая метрика - корень от средней квадратической ошибки. В свою очередь средняя квадратическая ошибка - это величина равная сумме квадратов отклонений (разница между предсказанием и фактической величиной целевого признака), разделенной на количество объектов. Чем точнее модель, тем больше значение RMSE будет стремиться к нулю. 

Оценим качество работы моделей согласно значениям метрики RMSE.

Для первой (учитывающей все признаки):

In [25]:
evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})

69547.08641331746

И для второй (учитывающей только количественные признаки):

In [26]:
evaluator.evaluate(predictions2, {evaluator.metricName: "rmse"})

70338.98028399626

Как видим, согласно значениям метрики RMSE первая модель работает точнее.

###  MAE:

Среднее абсолютное значение или MAE - это величина, равная сумме всех модулей отклонений (разница между предсказанием и фактическим значением целевого признака), разделенная на количество объектов в выборке.

Соответственно, для модели, которая идеально работает, MAE будет равен 0. Поэтому, чем меньше будет значение этой метрики, тем точнее модель. 

Оценим качество работы моделей:

Для первой (учитывающей все признаки):

In [27]:
evaluator.evaluate(predictions, {evaluator.metricName: "mae"})

50655.72889574194

И для второй (учитывающей только количественные признаки):

In [28]:
evaluator.evaluate(predictions2, {evaluator.metricName: "mae"})

51552.85923933801

Как видим, согласно значениям метрики MAE первая модель работает точнее.

###  R2: 

Коэффициент детерминации (R2) равен значению равному:

1 -  доля среднеквадратической ошибки модели от MSE среднего.

Это значит, что R2 равно единице, когда модель идеально предсказывает результат, а 0 - когда работает также, как и среднее. Соответственно, чем выше R2, тем точнее модель.

Подсчитаем значение R2 для каждой из моделей:

Для первой (учитывающей все признаки):

In [29]:
evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

0.6389097095403226

И для второй (учитывающей только количественные признаки):

In [30]:
evaluator.evaluate(predictions2, {evaluator.metricName: "r2"})

0.6306398264505922

Как видим, согласно значениям метрики R2 первая модель работает точнее.

### Выводы:

В рамках данной работы мы:

* Инициализировали локальную Spark-сессию
* Провели предобработку данных, используя методы pySpark 
* Исследовали данные на наличие пропусков и избавились от них
* Преобразовали колонку с категориальными значениями техникой One hot encoding.
* Провели стандартизацию количественных признаков

Построив две модели линейной регрессии на разных наборах данных:

1) используя все данные из файла;

2) используя только числовые переменные, исключив категориальные.


мы выяснили, что **согласно всем трем метрикам  RMSE, MAE и R2 точнее работает первая модель, использующая все данные из файла.**
